# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
vacation_df = pd.read_csv('Weather Output/fixed_weather.csv')
vacation_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Beloha,-25.1667,45.0500,72.84,90,0,8.43,MG,1645494543
1,1,Mgandu,-5.9500,34.1333,59.77,96,100,3.33,TZ,1645494596
2,3,Punta Arenas,-53.1500,-70.9167,59.11,55,40,4.61,CL,1645494597
3,4,Mataura,-46.1927,168.8643,50.65,98,100,6.87,NZ,1645494597
4,5,Jamestown,42.0970,-79.2353,43.92,70,100,1.99,US,1645494597
...,...,...,...,...,...,...,...,...,...,...
551,609,Crotone,39.0851,17.1178,42.93,75,46,13.56,IT,1645495523
552,610,Karratha,-20.7377,116.8463,92.57,45,68,9.28,AU,1645495523
553,611,Coos Bay,43.3665,-124.2179,42.93,98,73,11.01,US,1645495261
554,612,Akot,21.0964,77.0586,70.41,24,0,2.53,IN,1645495523


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
locations = vacation_df[['Lat', "Lng"]].astype(float)
humidity = vacation_df['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
ideal_temp = vacation_df['Max Temp'] < 75
ideal_humidity = vacation_df['Humidity'] < 50
ideal_cloudiness = vacation_df['Cloudiness'] > 40


destination_df = vacation_df.loc[ideal_temp & ideal_humidity & ideal_cloudiness]
df_len = len(destination_df['City'])

destination_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,38,Gazli,40.1333,63.4500,45.34,42,100,8.90,UZ,1645494611
33,40,Saint George,37.1041,-113.5841,54.19,35,100,11.99,US,1645494559
105,117,Marzuq,14.4000,46.4667,57.27,37,87,3.83,YE,1645494755
124,138,Tierra Nueva,21.6833,-100.5667,65.25,44,51,3.67,MX,1645494760
166,183,Sukumo,32.9333,132.7333,45.77,47,45,17.34,JP,1645494834
175,193,Shahr-e Bābak,30.1165,55.1186,40.05,45,85,1.21,IR,1645494837
176,194,Tiznit Province,29.5833,-9.5000,64.89,25,97,11.43,MA,1645494837
193,214,Sirsa,29.5333,75.0167,57.07,44,89,13.87,IN,1645494903
200,221,Sterling,39.0062,-77.4286,54.37,45,100,3.00,US,1645494906
282,310,Buta,2.7858,24.7300,64.65,42,100,2.66,CD,1645495052


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#test_lat = vacation_df.iloc[0,2]
#test_lng = vacation_df.iloc[0,3]
#test_input = 'hotel'
#test_url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={test_input}&inputtype=textquery&language=en&fields=name,geometry&locationbias=circle:5000@{test_lat},{test_lng}&key={g_key}'

#response = requests.get(test_url).json()


In [18]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "candidates": [
        {
            "geometry": {
                "location": {
                    "lat": 29.5004109,
                    "lng": -98.7171206
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.50182382989272,
                        "lng": -98.71577957010729
                    },
                    "southwest": {
                        "lat": 29.49912417010728,
                        "lng": -98.71847922989272
                    }
                }
            },
            "name": "Hotels"
        }
    ],
    "status": "OK"
}


In [20]:
input = 'hotel'

hotel_names = []
hotel_lat = []
hotel_lng = []

for x in range(df_len):
    lat = vacation_df.iloc[x,2]
    lng = vacation_df.iloc[x,3]
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={input}&inputtype=textquery&language=en&fields=name,geometry&locationbias=circle:5000@{lat},{lng}&key={g_key}'

    response = requests.get(url).json()
    hotel_names.append(response['candidates'][0]['name'])
    hotel_lat.append(response['candidates'][0]['geometry']['location']['lat'])
    hotel_lng.append(response['candidates'][0]['geometry']['location']['lng'])
    

In [23]:
hotel_df = destination_df

hotel_df['Hotel Name'] = hotel_names
hotel_df['Latitude'] =  hotel_lat
hotel_df['Longitude'] = hotel_lng

hotel_df


C:\Users\nalle\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\nalle\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\nalle\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be s

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Latitude,Longitude
31,38,Gazli,40.1333,63.4500,45.34,42,100,8.90,UZ,1645494611,Hotels,29.500411,-98.717121
33,40,Saint George,37.1041,-113.5841,54.19,35,100,11.99,US,1645494559,St. Gaspar Hostel,-5.698433,34.501716
105,117,Marzuq,14.4000,46.4667,57.27,37,87,3.83,YE,1645494755,Hotel Almasur Punta Arenas,-53.160553,-70.910937
124,138,Tierra Nueva,21.6833,-100.5667,65.25,44,51,3.67,MX,1645494760,Croydon Lodge Hotel,-46.084314,168.937334
166,183,Sukumo,32.9333,132.7333,45.77,47,45,17.34,JP,1645494834,La Quinta Inn & Suites by Wyndham Jamestown,42.096091,-79.244190
175,193,Shahr-e Bābak,30.1165,55.1186,40.05,45,85,1.21,IR,1645494837,Hotels,29.500411,-98.717121
176,194,Tiznit Province,29.5833,-9.5000,64.89,25,97,11.43,MA,1645494837,Holiday Inn Express & Suites East Greenbush(Al...,42.632346,-73.701767
193,214,Sirsa,29.5333,75.0167,57.07,44,89,13.87,IN,1645494903,Blue Lagoon Hotel,-32.983472,27.951098
200,221,Sterling,39.0062,-77.4286,54.37,45,100,3.00,US,1645494906,Kauai Shores Hotel,22.052908,-159.329498
282,310,Buta,2.7858,24.7300,64.65,42,100,2.66,CD,1645495052,Gostinitsa Khutorok,56.064812,97.621350


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))